In [135]:
from aeon.utils.windowing import sliding_windows
import pandas as pd
from aeon.distances import msm_pairwise_distance
from sklearn.cluster import DBSCAN,KMeans
from aeon.clustering import TimeSeriesKMeans
import numpy as np

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [88]:
events = pd.read_csv('dodgers+loop+sensor/Dodgers.events', header=None)
df = pd.read_csv('dodgers+loop+sensor/Dodgers.data', header=None)

In [89]:
df.rename(columns={0: 'date', 1: 'car_count'}, inplace=True)
df = df[df['car_count'] != -1]
df['y']=0


In [90]:
events['start_date']=pd.to_datetime(events[0].astype(str) + ' ' + events[1].astype(str))
events['end_date']=pd.to_datetime(events[0].astype(str) + ' ' + events[2].astype(str))


/tmp/ipykernel_88955/1274556392.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events['start_date']=pd.to_datetime(events[0].astype(str) + ' ' + events[1].astype(str))
/tmp/ipykernel_88955/1274556392.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events['end_date']=pd.to_datetime(events[0].astype(str) + ' ' + events[2].astype(str))


In [91]:
events

,0,1,2,3,4,5,start_date,end_date
0,04/12/05,13:10:00,16:23:00,55892,San Francisco,W 9-,2005-04-12 13:10:00,2005-04-12 16:23:00
1,04/13/05,19:10:00,21:48:00,46514,San Francisco,W 4-,2005-04-13 19:10:00,2005-04-13 21:48:00
2,04/15/05,19:40:00,21:48:00,51816,San Diego,W 4-0,2005-04-15 19:40:00,2005-04-15 21:48:00
3,04/16/05,19:10:00,21:52:00,54704,San Diego,W 8-3,2005-04-16 19:10:00,2005-04-16 21:52:00
4,04/17/05,13:10:00,15:31:00,53402,San Diego,W 6-0,2005-04-17 13:10:00,2005-04-17 15:31:00
...,...,...,...,...,...,...,...,...
76,09/25/05,13:10:00,16:06:00,37846,Pittsburgh,W 9-2,2005-09-25 13:10:00,2005-09-25 16:06:00
77,09/26/05,19:10:00,22:27:00,36397,Pittsburgh,W 9-4,2005-09-26 19:10:00,2005-09-26 22:27:00
78,09/27/05,19:10:00,21:33:00,42604,Arizona,L 2-0,2005-09-27 19:10:00,2005-09-27 21:33:00
79,09/28/05,19:10:00,21:58:00,46424,Arizona,L 4-3,2005-09-28 19:10:00,2005-09-28 21:58:00


In [92]:
# df['date'] = pd.to_datetime(df['date'], format=r'%mm%dd%YY %H%M')
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%m/%d/%Y %H:%M')
df['date'] = pd.to_datetime(df['date'])


In [93]:
for i in range(len(events)):
    event_start = events['start_date'][i]
    event_end = events['end_date'][i]
    df.loc[(df['date'] >= event_start) & (df['date'] <= event_end), 'y'] = 1

In [94]:
y= df['y']
x= df['car_count']
x=x.to_numpy()

In [45]:
windows, _ = sliding_windows(x, window_size=12,stride=12)

In [18]:
distance_matrix = msm_pairwise_distance(windows)

In [49]:
clst= TimeSeriesKMeans(distance="msm",n_clusters=2)

In [50]:
clst.fit(windows)

TimeSeriesKMeans(n_clusters=2)

In [51]:
centers = clst.cluster_centers_

In [52]:
centers

array([[[12,  9, 10,  9,  9,  9,  9,  9, 10, 10, 10, 10]],

       [[30, 32, 23, 35, 27, 24, 37, 29, 29, 31, 29, 30]]])

In [57]:
predictions = clst.predict(windows)

In [69]:
windows[10]

array([20, 26, 33, 32, 29, 38, 36, 33, 30, 27, 31, 44])

In [73]:
values=windows.flatten().shape

In [125]:
all_preds = np.repeat(predictions,12)
all_preds=list(all_preds)
all_preds.append(0)

all_preds_swap = [1 if pred==0 else 0 for pred in all_preds]



In [148]:
df['predictions'] = all_preds

In [149]:
df

,date,car_count,y,predictions
379,2005-04-11 07:35:00,23,0,1
380,2005-04-11 07:40:00,42,0,1
381,2005-04-11 07:45:00,37,0,1
382,2005-04-11 07:50:00,24,0,1
383,2005-04-11 07:55:00,39,0,1
...,...,...,...,...
50109,2005-09-30 23:45:00,14,0,0
50110,2005-09-30 23:50:00,12,0,0
50111,2005-09-30 23:55:00,8,0,0
50112,2005-10-01 00:00:00,13,0,0


In [150]:
f1_score(df['y'],df['predictions'])

np.float64(0.14492753623188406)

In [151]:
precision_score(df['y'],df['predictions'])

np.float64(0.08032852564102565)

In [152]:
recall_score(df['y'],df['predictions'])

np.float64(0.7401255075673681)